<a href="https://colab.research.google.com/github/4060e059/funnyimage/blob/main/%E5%9C%96%E7%89%87%E9%A2%A8%E6%A0%BC%E8%BD%89%E6%8F%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
import numpy as np
import PIL.Image
import time
import functools

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:

# 設定你的照片
cpathname="myphoto4"#@param {type: "string"}
cpath="https://raw.githubusercontent.com/4060e059/image/main/IMAG011.jpg"#@param {type: "string"}
#cpath="https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg"#@param {type: "string"}

# 設定要使用的藝術家風格
spathname="mystylephoto4"#@param {type: "string"}
spath="https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg"#@param {type: "string"}

# 設定
content_path = tf.keras.utils.get_file(cpathname, cpath)
style_path = tf.keras.utils.get_file(spathname,spath)
#範例圖片:  https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg
#範例圖片:  https://pic.pimg.tw/volvo2266/1483538279-633293117.jpg

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

In [ ]:

# 使用Tensorflow HUB 進行 Transfer Learning(遷移學習)
import tensorflow_hub as hub

hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

#顯示最後成果
tensor_to_image(stylized_image)